## How to contribute a slicing function for Xingu

Slicing functions define a spatial and temporal area that can be used to evaluate automatically generated models in Xingu

### Login GEE API

In [100]:
# Import the Earth Engine API and initialize it.
import ee
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize()

UsageError: Line magic function `%tensorflow_version` not found.


## Specify base maps

In [20]:
# Use these bands for prediction.
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
# Use Landsat 8 surface reflectance data.
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
ndvi = ee.ImageCollection('LANDSAT/LC08/C01/T1_32DAY_NDVI')
hansen = ee.Image("UMD/hansen/global_forest_change_2018_v1_6")
#l7sr = ee.ImageCollection('LANDSAT/LC07/C01/T1_SR')
#sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

## get nighttime lights
nightlight_col = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG')

def maskS2clouds(image):
  qa = image.select('QA60');
  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10;
  cirrusBitMask = 1 << 11;

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0));

  return image.updateMask(mask).divide(10000);
                                                 
                                                          
## Cloud masking function.
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask).select(bands).divide(10000)



## Get median composites for years 2013-2019

In [21]:
images_per_year = dict()
for year in range(2013, 2019):
    print(year)
    images_per_year[year] = {'ls8sr':  l8sr.filterDate('{}-01-01'.format(year), '{}-12-31'.format(year)).map(maskL8sr).median()}
    images_per_year[year]['nightlight'] = nightlight_col.filterDate('{}-01-01'.format(year), '{}-12-31'.format(year)).median()
    images_per_year[year]['ndvi'] = ndvi.filterDate('{}-01-01'.format(year), '{}-12-31'.format(year)).median()
    lossYear = hansen.select(['lossyear'])
    images_per_year[year]['hansen'] = lossYear.gt(year - 2000)


2013
2014
2015
2016
2017
2018


## Get all available scenario regions

In [59]:
import os
import geojson
base_dir = os.getcwd()
all_geojson_paths = [os.path.join(base_dir, x) for x in os.listdir(base_dir) if x.endswith('.geojson')]
all_geojsons = []
for geojson_path in all_geojson_paths:
    with open(geojson_path) as f:
        gj = geojson.load(f)
        all_geojsons.append(gj)
print('found {} scenario regions'.format(len(all_geojsons)))
#all_geojsons = sorted(all_geojsons)
import random
random.seed(123)
random.shuffle(all_geojsons)
train_range = round(0.7 * len(all_geojsons))
val_range = round(0.8 * len(all_geojsons))
train_regions = all_geojsons[:train_range]
val_regions = all_geojsons[train_range:val_range]
test_regions = all_geojsons[val_range:]


found 16 scenario regions


In [60]:
#get sample train region
#train_region = train_regions[0]

In [61]:
def get_coordinates_for_region(region):
    region = train_region["coordinates"][0]
    region_inv = [[x[1],x[0]] for x in region]
    x_center = sum(x[0] for x in region_inv) / len(region_inv)
    y_center = sum(x[1] for x in region_inv) / len(region_inv)
    edges = region_inv
    center_coords = [x_center, y_center]
    return edges, center_coords



## This is how we visualize it in colab

In [74]:
def get_scaled_img(image, target_geometry):
    minMax = image.reduceRegion(reducer= ee.Reducer.minMax(), geometry= target_geometry,scale= 30, maxPixels= 10e9,   # tileScale: 16
    );
    def scale(name):
        name = ee.String(name);
        #print('band name: {}'.format(name))
        band = image.select(name);
        
        return band.unitScale(ee.Number(minMax.get(name.cat('_min'))), ee.Number(minMax.get(name.cat('_max'))))
                    # eventually multiply by 100 to get range 0-100
                    #.multiply(100); 
    scaled_bands = image.bandNames().map(scale)
    unitScale = ee.ImageCollection.fromImages(scaled_bands)
    unitScale_bands = unitScale.toBands().rename(image.bandNames());
    
#    meanDict = unitScale_bands.reduceRegion(reducer=  ee.Reducer.mean(), geometry =target_geometry, scale=30 )
#    minMaxDict = unitScale_bands.reduceRegion(reducer=  ee.Reducer.minMax(), geometry =target_geometry, scale=30 )
#    print(meanDict.getInfo())
#    print(minMaxDict.getInfo())
    
    return unitScale_bands

def maskGeometry(image, geometry):
   mask =  ee.Image.constant(1).clip(geometry).mask()
   return image.updateMask(mask)

In [75]:

import folium

def display_img(image, center_location, value_name, mask_region):
    mapIdDict = image.getMapId({'bands': [value_name], 'min': 0.0, 'max': 1})
    folium_map = folium.Map(location=center_location)
    folium.TileLayer(
        tiles=mapIdDict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        overlay=True,
        name='median composite',
      ).add_to(folium_map)
    folium_map.add_child(folium.LayerControl())

    line_color='red'
    fill_color='red'
    weight=2
    text='Selected Region'
    
    folium_map.add_child(folium.vector_layers.Polygon(locations=mask_region, color=line_color, fill_color=fill_color,
                                                  weight=weight, popup=(folium.Popup(text))))

    return folium_map

### We use folium to visualize the map

In [76]:
def compute_and_display_yeardiff(images_per_year, year, img_type, value_name, region):
    image = images_per_year[year][img_type]
    img_cur = maskGeometry(images_per_year[year][img_type], region)
    img_old = maskGeometry(images_per_year[year-1][img_type], region)
    imgs_diff = img_cur.select(value_name).subtract(img_old.select(value_name))
    #normalized_diff
    #imgs_normdiff = imgs_diff.divide(img_cur.select(value_name).add(img_old.select(value_name)))

    #Define a kernel.
    #kernel = ee.Kernel.circle(radius= 15);
    #dilated = maskGeometry(imgs_normdiff.focal_max(kernel= kernel, iterations= 2), region)
    #roi_image = maskGeometry(image, region)
    scaled_img = get_scaled_img(imgs_diff, region)
    edges, center_coords = get_coordinates_for_region(region)
    
    folium_map = display_img(scaled_img, center_location=center_coords, value_name=value_name, mask_region=edges)
    return folium_map, scaled_img

def compute_and_display_year(images_per_year, year, img_type, value_name, region):
    image = images_per_year[year][img_type]
    img_cur = maskGeometry(images_per_year[year][img_type], region)
    scaled_img = get_scaled_img(img_cur, region)
    edges, center_coords = get_coordinates_for_region(region)
    folium_map = display_img(scaled_img, center_location=center_coords, value_name=value_name, mask_region=edges)
    return folium_map, scaled_img


## Display year difference for demo  

In [78]:
year = 2016
train_region = train_regions[0]
#folium_map, scaled_img = display_year(images_per_year, year=year, img_type='hansen', value_name='lossyear', train_region=train_region)
folium_map, scaled_yeardiff = compute_and_display_yeardiff(images_per_year, year=year, img_type='ndvi', value_name='NDVI', region=train_region)
folium_map

## Collect "year difference" features for NDVI and nightlight

In [82]:
#print(folium.__version__)
scaled_features = dict()
gt_labels = dict()

datasets = dict()

set_types = ['train', 'val', 'test']
set_regions = [train_regions, val_regions, test_regions]

for set_type, regions in zip(set_types[:1], set_regions[:1]):
    for i, region in enumerate(regions):
        print('collecting features for region {} out of {}'.format(i, len(regions)))
        for year in range(2015, 2018):
            if year not in scaled_features:
                scaled_features[year] = dict()
            for img_type, value_name in zip(['ndvi', 'nightlight'], ['NDVI', 'avg_rad']):
                if img_type not in scaled_features[year]:
                    scaled_features[year][img_type] = dict()

                _, scaled_yeardiff = display_yeardiff(images_per_year, year=year, img_type=img_type, value_name=value_name, region=region)
                _, scaled_img = compute_and_display_year(images_per_year, year=year, img_type=img_type, value_name=value_name, region=region)
                scaled_features[year][img_type] = {'default': scaled_img, 'yeardiff': scaled_yeardiff}
            #yeardiff
            folium_map, scaled_img = compute_and_display_year(images_per_year, year=year, img_type='hansen', value_name='lossyear', region=region)
            gt_labels[year] = scaled_img
            datasets[set_type] = {'features': scaled_features, 'labels': gt_labels}




collecting features for region 0 out of 11
collecting features for region 1 out of 11
collecting features for region 2 out of 11
collecting features for region 3 out of 11
collecting features for region 4 out of 11
collecting features for region 5 out of 11
collecting features for region 6 out of 11
collecting features for region 7 out of 11
collecting features for region 8 out of 11
collecting features for region 9 out of 11
collecting features for region 10 out of 11


    ## Inspect training and validation polygons

In [83]:
trainingPolys = ee.FeatureCollection(train_regions)
evalPolys = ee.FeatureCollection(test_regions)
valPolys = ee.FeatureCollection(val_regions)

polyImage = ee.Image(0).byte().paint(trainingPolys, 1).paint(valPolys, 2).paint(evalPolys, 3)
polyImage = polyImage.updateMask(polyImage)

mapid = polyImage.getMapId({'min': 1, 'max': 3, 'palette': ['red', 'green', 'blue']})
map = folium.Map(zoom_start=5)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='training polygons',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [108]:
import tensorflow as tf

##Create a demo feature from the training data
featureStack = ee.Image.cat([
     datasets['train']['features'][2016]['ndvi']['default'],
     datasets['train']['features'][2016]['ndvi']['yeardiff'],
     datasets['train']['features'][2016]['nightlight']['default'],
     datasets['train']['features'][2016]['nightlight']['yeardiff']
]).float()

FEATURES = ['ndvi_default', 'ndvi_yeardiff', 'nightlight_default', 'nightlight_yeardiff']
BANDS = FEATURES

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for f in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

# Sizes of the training and evaluation datasets.
TRAIN_SIZE = 16000
EVAL_SIZE = 8000

# Specify model training parameters.
BATCH_SIZE = 16
EPOCHS = 10
BUFFER_SIZE = 2000
OPTIMIZER = 'SGD'
LOSS = 'MeanSquaredError'
METRICS = ['RootMeanSquaredError']

list = ee.List.repeat(1, KERNEL_SIZE)
lists = ee.List.repeat(list, KERNEL_SIZE)
kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, lists)

arrays = featureStack.neighborhoodToArray(kernel)


In [122]:
# Convert the feature collections to lists for iteration.
trainingPolysList = trainingPolys.toList(trainingPolys.size())
#evalPolysList = evalPolys.toList(evalPolys.size())

# Specify names locations for outputs in Cloud Storage. 
FOLDER = 'xingu-demo'
TRAINING_BASE = 'training_patches'
EVAL_BASE = 'eval_patches'

# These numbers determined experimentally.
n = 200 # Number of shards in each polygon.
N = 2000 # Total sample size in each polygon.

print('number of train polygons: {}'.format(trainingPolys.size().getInfo()))
# Export all the training data (in many pieces), with one task 
# per geometry.
for g in range(trainingPolys.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  for i in range(n):
    sample = arrays.sample(
      region = ee.Feature(trainingPolysList.get(g)).geometry(), 
      scale = 30, 
      numPixels = N / n, # Size of the shard.
      seed = i,
      tileScale = 8
    )
    geomSample = geomSample.merge(sample)
  
    desc = TRAINING_BASE + '_g' + str(g)
    task = ee.batch.Export.table.toDrive(
    collection = geomSample,
    description = desc, 
    fileNamePrefix = FOLDER + '/' + desc,
    fileFormat = 'TFRecord',
    selectors = FEATURES
    )
  task.start()

# Convert the feature collections to lists for iteration.
trainingPolysList = trainingPolys.toList(trainingPolys.size())
evalPolysList = evalPolys.toList(evalPolys.size())

# These numbers determined experimentally.
n = 200 # Number of shards in each polygon.
N = 2000 # Total sample size in each polygon.

# Export all the training data (in many pieces), with one task 
# per geometry.
for g in range(trainingPolys.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  for i in range(n):
    sample = arrays.sample(
      region = ee.Feature(trainingPolysList.get(g)).geometry(), 
      scale = 30, 
      numPixels = N / n, # Size of the shard.
      seed = i,
      tileScale = 8
    )
    geomSample = geomSample.merge(sample)
  
  desc = TRAINING_BASE + '_g' + str(g)
  task = ee.batch.Export.table.toDrive(
    collection = geomSample,
    description = desc, 
    fileNamePrefix = FOLDER + '/' + desc,
    fileFormat = 'TFRecord',
    selectors = FEATURES
  )
  task.start()

# Export all the evaluation data.
for g in range(evalPolys.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  for i in range(n):
    sample = arrays.sample(
      region = ee.Feature(evalPolysList.get(g)).geometry(), 
      scale = 30, 
      numPixels = N / n,
      seed = i,
      tileScale = 8
    )
    geomSample = geomSample.merge(sample)
  
  desc = EVAL_BASE + '_g' + str(g)
  task = ee.batch.Export.table.toDrive(
    collection = geomSample,
    description = desc, 
    fileNamePrefix = FOLDER + '/' + desc,
    fileFormat = 'TFRecord',
    selectors = FEATURES
  )
  task.start()



number of train polygons: 11


In [130]:
def parse_tfrecord(example_proto):
  """The parsing function.
  Read a serialized example into the structure defined by FEATURES_DICT.
  Args:
    example_proto: a serialized Example.
  Returns: 
    A dictionary of tensors, keyed by feature name.
  """
  return tf.io.parse_single_example(example_proto, FEATURES_DICT)


def to_tuple(inputs):
  """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
  Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
  Args:
    inputs: A dictionary of tensors, keyed by feature name.
  Returns: 
    A dtuple of (inputs, outputs).
  """
  inputsList = [inputs.get(key) for key in FEATURES]
  stacked = tf.stack(inputsList, axis=0)
  # Convert from CHW to HWC
  stacked = tf.transpose(stacked, [1, 2, 0])
  return stacked[:,:,:len(FEATURES)], stacked[:,:,len(FEATURES):]


def get_dataset(pattern):
  """Function to read, parse and format to tuple a set of input tfrecord files.
  Get all the files matching the pattern, parse and convert to tuple.
  Args:
    pattern: A file pattern to match in a Cloud Storage bucket.
  Returns: 
    A tf.data.Dataset
  """
  glob = tf.io.gfile.glob(pattern)
  print('glob: {}'.format(glob))
  #glob=[]
  dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
  dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)
  print(dataset)
  dataset = dataset.map(to_tuple, num_parallel_calls=5)
  return dataset

In [139]:
from google.colab import drive
drive.mount('/gdrive')

#TODO: this doesnt work locally

def get_training_dataset():
	"""Get the preprocessed training dataset
  Returns: 
    A tf.data.Dataset of training data.
  """
	#glob = 'gs://' +  FOLDER + '/' + TRAINING_BASE + '*'
	glob = '/gdrive/' +  FOLDER + '/' + TRAINING_BASE + '*'
	dataset = get_dataset(glob)
	dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
	return dataset

training = get_training_dataset()

#print(iter(training.take(1)).next())

KeyError: 'CLOUDSDK_CONFIG'